# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** K

**Names:**

* Kim Lan Phan Hoang
* Robin Lang

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

## Exercise 4.1: Pre-processing

### Pre-process the corpus to create bag-of-words representations of each document

### Explain which kinds of cleaning you implemented and why

### Print the terms in the pre-processed description of the IX class in alphabetical order.

## Exercise 4.2: Term-document matrix

### Construct an M×N term-document matrix X, where M is the number of terms and N is the number of documents. The matrix X should be sparse. You are not allowed to use libraries for this task.

### Print the 15 terms in the description of the IX class with the highest TF-IDF scores.

### Explain where the difference between the large scores and the small ones comes from.

## Exercise 4.3: Document similarity search

### Search for "markov chains" and "facebook".

### Display the top five courses together with their similarity score for each query.

### What do you think of the results? Give your intuition on what is happening.